<a href="https://colab.research.google.com/github/Mircus/Mining-Cointegration-Graphs-/blob/master/SHE_BERT_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl 
!pip install xlrd==1.2.0
from operator import index

import pandas as pd
from sklearn.model_selection import train_test_split

!pip install odfpy
# reads the dataset to be evaluated

#df_startups = pd.read_csv("/content/sample_data/valid.csv")
#df_startups['label']="lose"
import re

def createDescription( df_startups): 
    df_startups["FullDescription"] =                                  "  description:  "          +  df_startups["Description"]
    df_startups["FullDescription"] = df_startups["FullDescription"] + "  Verticals:   "           + df_startups["Verticals"]
    df_startups["FullDescription"] = df_startups["FullDescription"] + "  primaryindustrygroup: "  + df_startups["Primary Industry Group"] 
    df_startups["FullDescription"] = df_startups["FullDescription"] + "  financingstatus:   "     + df_startups["Financing Status"]
    df_startups["FullDescription"] = df_startups["FullDescription"] + "  businessstatus :   "     + df_startups["Business Status"]
    df_startups["FullDescription"] = df_startups["FullDescription"] + "  dealsynopsis :   "       + df_startups["Deal Synopsis"]
    df_startups["FullDescription"] = df_startups["FullDescription"] + "  investors  :   "         + df_startups["Investors"]
    df_raw = df_startups[["FullDescription"]].dropna()
    return df_raw

# instantiate model

In [2]:
!pip install transformers

from transformers import BertTokenizer, BertModel
from torch import nn

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer   




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.8 MB/s eta 0:00:00


get data in dataset 

In [3]:
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'lose':0,
          'win':1        
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [labels[label] for label in df['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['FullDescription']]

    def classes(self):
        return self.labels
    def __len__(self):
        return len(self.labels)
    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])
    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]
    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y


In [4]:
def eval(model, data):   
    y_pred = []
    test = Dataset(data)
    dataloader = torch.utils.data.DataLoader(test, batch_size=2)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
    
    with torch.no_grad():
        for input, label in   dataloader:
              test_label = label.to(device)
              mask = input['attention_mask'].to(device)
              input_id =input['input_ids'].squeeze(1).to(device)
              output = model(input_id, mask)
              output1 = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
              y_pred.extend(output1) # Save Prediction             
              #label = test_label.data.cpu().numpy()       

    return  y_pred

In [ ]:
import numpy as np

df_raw= createDescription(df_startups)
#read model from file 
df_startups['label']="lose"
PATH_TO_MODEL ="/content/shemodel.pth"
model = BertClassifier()
model.load_state_dict(torch.load(PATH_TO_MODEL, map_location=torch.device('cpu')))
results = eval(model, df_startups)
print(results) 


In [ ]:
!pip install streamlit

In [5]:
%%writefile streamlit_app.py 

!pip install transformers

import streamlit as st
import torch
import pandas as pd
from transformers import  pipeline
from transformers import BertTokenizer, BertModel
from torch import nn

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer   


tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'lose':0,
          'win':1        
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [labels[label] for label in df['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['FullDescription']]

    def classes(self):
        return self.labels
    def __len__(self):
        return len(self.labels)
    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])
    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]
    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

    def createDescription( df_startups): 
     df_startups["FullDescription"] =                                  "  description:  "          +  df_startups["Description"]
     df_startups["FullDescription"] = df_startups["FullDescription"] + "  Verticals:   "           + df_startups["Verticals"]
     df_startups["FullDescription"] = df_startups["FullDescription"] + "  primaryindustrygroup: "  + df_startups["Primary Industry Group"] 
     df_startups["FullDescription"] = df_startups["FullDescription"] + "  financingstatus:   "     + df_startups["Financing Status"]
     df_startups["FullDescription"] = df_startups["FullDescription"] + "  businessstatus :   "     + df_startups["Business Status"]
     df_startups["FullDescription"] = df_startups["FullDescription"] + "  dealsynopsis :   "       + df_startups["Deal Synopsis"]
     df_startups["FullDescription"] = df_startups["FullDescription"] + "  investors  :   "         + df_startups["Investors"]
     df_raw = df_startups[["FullDescription"]].dropna()
     return df_raw

def load_model:
  # Define the model name and path
  model_name = "my-custom-bert-model"
  PATH_TO_MODEL ="/content/shemodel.pth"
  # Load the saved model 
  @st.cache_resource
  model = BertClassifier()
# model.load_state_dict(torch.load(PATH_TO_MODEL, map_location=torch.device('cpu')))

# Create a file uploader widget
file = st.file_uploader("Upload your startups csv file", type=["csv"])

# Define the app

def main():
    # Load the model
    model = load_model()

    st.title("Custom BERT SHE evaluator  alpha 1.0")
    # If a file was uploaded
    if file is not None:
        # Read the file into a Pandas DataFrame
        df = pd.read_csv(file)    
        #transform df to maake it usabl by model 
        df_raw = createDescription( df)
        results = eval(model, df_raw)

        # Add a column to the DataFrame for user results
        #df['target'] = df_`raw['text'].apply(eval_startup)

        # Display the DataFrame
        st.write(results)


if __name__ == "__main__":
    main()    


Writing streamlit_app.py


In [6]:
!ls

sample_data  streamlit_app.py


In [ ]:
#!streamlit run test_app.py --server.port 8501 
!streamlit run test_app.py 


In [ ]:
!ps -Al | grep streamlit